In [123]:
import pandas as pd
import json
import multiprocessing
from multiprocessing import Pool
import numpy as np
import math

In [10]:
def get_data_and_header(clean=False):
    if clean:
        df = pd.read_csv('../Data/transformed_2017.csv')
        return df, list(df)
    df = pd.read_csv('../Data/usa_00021.csv')
    drop_list = ['DATANUM', 'SERIAL', 'CBSERIAL', 'PERNUM', 'RACED', 'BPLD', 'LANGUAGED', 'OCC', 'IND', 'HHWT']
    df.drop(drop_list, axis=1, inplace=True)
    header = list(df)
    return df, header

def read_json(f_name):
    with open(f_name) as j:  
        codebook = json.load(j)
    return codebook

In [186]:
def is_numerical(h):
    return h in ['INCOME', 'HHWT', 'FARMPROD', 'MORTAMT1', 'PROPINSR', 'RENTGRS', 'CONDOFEE', 'MOBLHOME'
                ,'COSTELEC', 'COSTGAS', 'COSTWATR', 'COSTFUEL', 'HHINCOME', 'VALUEH', 'PERWT',
                'MOMLOC', 'POPLOC', 'BIRTHYR', 'RACED', 'YRIMMIG', 'YRSUSA1', 'OCC', 'IND', 'INCTOT',
                'FTOTINC', 'INCSS', 'INCWELFR', 'INCRETIR', 'POVERTY', 'OCCSCORE', 'SEI', 'HWSEI',
                'PRESGL', 'PRENT', 'MOVEDIN', 'RIDERS', 'TRANTIME', 'DEPARTS', 'ROOMS']

def is_null_numerical(each):
    
    each = list(str(each))
    
    all_nines = []
    all_nines_minus_one = []
    
    for e in each[:-1]:
        all_nines.append('9')
        all_nines_minus_one.append('9')
    
    all_nines.append('9')
    
    if len(each) == 1:
        return False
    
    if (all_nines == each) or (all_nines_minus_one == each[:-1]):
        return True
    
    return False


def is_null_categorical(each):

    missing_value_indicators = ['not identified', 'N/A']

    if type(each) == str:    
        for i in missing_value_indicators:
            if i in each:
                return True
            
    elif math.isnan(each):
        return True

    return False
   

def find_nulls(row):
    '''
    Rather than trying to return a totally new data structure, why not just standardize
        On replacing all nulls with nan
        Then using pandas data frame analytics
        To check them out
    '''
    
    new_row = list(row[:])
    
    for idx, each in enumerate(row):
        
        is_null = False
        
        h = header[idx]
        
        if h == 'YEAR':
            continue

        elif is_numerical(h) and is_null_numerical(each):
            new_row[idx] = 0
            
        elif is_null_categorical(each):
            new_row[idx] = 0
            
    return new_row

def magic_loop(rows):
    pool = Pool(processes=multiprocessing.cpu_count())
    transformed_rows= pool.map(find_nulls, rows)
    pool.close() 
    pool.join()
    df = pd.DataFrame(transformed_rows, columns = header)
    return df

transformed_df = magic_loop(rows)

In [164]:
codes = read_json('../Data/codebook.json')
df, header = get_data_and_header(clean=True)
rows = df.as_matrix()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (7,8,9,11,13,14,26,28,29,30,31,32,37,38,46,47,71) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [188]:
transformed_df.shape

(3190040, 110)

In [189]:
transformed_df.to_csv('../Data/transformed_missing_replaced_with_zero.csv', index=False)

In [190]:
!aws s3 cp '../Data/transformed_missing_replaced_with_zero.csv' s3://ep-state-data-raw/2017-Cleaned/

upload: ../Data/transformed_missing_replaced_with_zero.csv to s3://ep-state-data-raw/2017-Cleaned/transformed_missing_replaced_with_zero.csv
